In [1]:

import scipy.io as sio
import urllib.request 
import os.path
datamat = sio.loadmat('data/INSECTS/data.mat')

In [22]:
# batch download tsvs that are present in the original dataset
str = ""
for index, bold_id in enumerate(datamat['bold_ids']):
    str+=bold_id[0][0]
    if len(str)>6000:
        url = f"http://v3.boldsystems.org/index.php/API_Public/specimen?ids={str}&format=tsv"
        fname = f"csvs_a_pezzi/{index}.tsv"
        str = ""
        if not os.path.isfile(fname):
            try:
                urllib.request.urlretrieve(url,fname )
            except:
                print(len(url))
                break
    else:
        str+='|'
url = f"http://v3.boldsystems.org/index.php/API_Public/specimen?ids={str}&format=tsv"
fname = f"csvs_a_pezzi/{index}.tsv"
if not os.path.isfile(fname):
    try:
        urllib.request.urlretrieve(url,fname )
    except:
        print(len(url))

In [2]:
import pandas as pd
from os import listdir
import os
df = pd.DataFrame()
partial = 0
for f in listdir("csvs_a_pezzi_unix/"):
    temp_df = pd.read_csv(f"csvs_a_pezzi_unix/{f}", sep='\t',encoding='latin')
    partial += temp_df.shape[0]
    df = pd.concat([df,temp_df])

df.shape

(27362, 51)

In [3]:
df =df.dropna(subset=['image_urls'])
#print(df['image_urls'])
for i,row in df.iterrows():
    #print(type(row['image_urls']))
    if isinstance(row['image_urls'],float):
        print(i)
        


In [11]:
df.columns

Index(['processid', 'sampleid', 'recordID', 'catalognum', 'fieldnum',
       'institution_storing', 'bin_uri', 'phylum_taxID', 'phylum_name',
       'class_taxID', 'class_name', 'order_taxID', 'order_name',
       'family_taxID', 'family_name', 'subfamily_taxID', 'subfamily_name',
       'genus_taxID', 'genus_name', 'species_taxID', 'species_name',
       'subspecies_taxID', 'subspecies_name', 'identification_provided_by',
       'voucher_type', 'tissue_type', 'collectors', 'collectiondate',
       'lifestage', 'sex', 'reproduction', 'extrainfo', 'notes', 'lat', 'lon',
       'coord_source', 'coord_accuracy', 'country', 'province', 'region',
       'exactsite', 'image_ids', 'image_urls', 'copyright_licenses',
       'trace_ids', 'trace_links', 'run_dates', 'sequencing_centers',
       'directions', 'seq_primers', 'marker_codes'],
      dtype='object')

In [4]:
urls_list = []
print(df.shape)
for i,row in df.iterrows():
    for url in row['image_urls'].split('|'):
        urls_list.append(url)
#print(urls_list)
missing_urls = []

(27359, 51)


In [14]:
len(missing_urls)
len(urls_list)

33626

In [15]:
output_folder = 'image_dataset/'
import requests
from time import sleep
def save_image_from_url(url, output_folder):
    if os.path.isfile(os.path.join(output_folder, url[url.rfind('/')+1:])):
       return

    image = requests.get(url)
    output_path = os.path.join(
        output_folder, url[url.rfind('/')+1:]
    )
    with open(output_path, "wb") as f:
        f.write(image.content)
        sleep(1)

missing_urls = []
while urls_list:
    url = urls_list.pop()
    try:
        #decomment to use
        #save_image_from_url(url,output_folder)
    except Exception as e:
        print("fail",e)
        missing_urls.append(url)
        
        

fail HTTPConnectionPool(host='v3.boldsystems.org', port=80): Max retries exceeded with url: /pics/GRAFW/bioug02036-b07+1328647578.JPG (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7eb72b1d6f60>, 'Connection to v3.boldsystems.org timed out. (connect timeout=None)'))
fail HTTPConnectionPool(host='v3.boldsystems.org', port=80): Max retries exceeded with url: /pics/GRAFW/bioug02036-b02+1328647636.JPG (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7eb72b1d7050>, 'Connection to v3.boldsystems.org timed out. (connect timeout=None)'))


In [5]:
missing_urls

[]

In [6]:
df.head()

,processid,sampleid,recordID,catalognum,fieldnum,institution_storing,bin_uri,phylum_taxID,phylum_name,class_taxID,...,image_ids,image_urls,copyright_licenses,trace_ids,trace_links,run_dates,sequencing_centers,directions,seq_primers,marker_codes
0,ANICL062-10,10ANIC-08059,1652746,10ANIC-08059,,Australian National Insect Collection,BOLD:AAE3294,20,Arthropoda,82,...,917766,http://v3.boldsystems.org/pics/ANICL/10ANIC_08...,CreativeCommons - Attribution,2603931|2603908|2603881|2603843,http://trace.boldsystems.org/traceIO/bold.org/...,2010-12-16 00:05:06|2010-12-15 23:19:19|2010-1...,Centre for Biodiversity Genomics|Centre for Bi...,R|F|F|R,LepR1|MLepF1|LepF1|MLepR1,COI-5P|COI-5P|COI-5P|COI-5P
1,BCHYM11106-15,BC ZSM HYM 24216,6295592,BC ZSM HYM 24216,,"SNSB, Zoologische Staatssammlung Muenchen",BOLD:AAM4014,20,Arthropoda,82,...,2686021,http://v3.boldsystems.org/pics/BCHYM/BC_ZSM_HY...,CreativeCommons - Attribution Non-Commercial S...,8474808|8474713,http://trace.boldsystems.org/traceIO/bold.org/...,2015-10-05 14:30:49|2015-10-06 05:32:19,Centre for Biodiversity Genomics|Centre for Bi...,R|F,LepR1|LepF1,COI-5P|COI-5P
2,BCHYM7643-15,BC-ZSM-HYM-27761-D07,5806393,BC-ZSM-HYM-27761-D07,BC-ZSM-HYM-27761-D07,"SNSB, Zoologische Staatssammlung Muenchen",BOLD:AAZ8135,20,Arthropoda,82,...,2589871,http://v3.boldsystems.org/pics/BCHYM/BC-ZSM-HY...,CreativeCommons - Attribution Non-Commercial S...,7816517|7816422,http://trace.boldsystems.org/traceIO/bold.org/...,2015-05-04 11:31:35|2015-05-04 12:34:37,Centre for Biodiversity Genomics|Centre for Bi...,F|R,LepF1|LepR1,COI-5P|COI-5P
3,BLPBC407-07,06-SRNP-109430,568126,,06-SRNP-109430,Area de Conservacion Guanacaste,BOLD:AAA3052,20,Arthropoda,82,...,246186,http://v3.boldsystems.org/pics/BLPBC/06-SRNP-1...,CreativeCommons - Attribution Non-Commercial S...,742969|742875,http://trace.boldsystems.org/traceIO/bold.org/...,2007-08-30 07:31:59|2007-08-30 06:00:49,Centre for Biodiversity Genomics|Centre for Bi...,R|F,LepR1|LepF1,COI-5P|COI-5P
4,BLPBE439-07,07-SRNP-101378,623757,,07-SRNP-101378,Area de Conservacion Guanacaste,BOLD:AAA3052,20,Arthropoda,82,...,226489,http://v3.boldsystems.org/pics/BLPBE/07-SRNP-1...,CreativeCommons - Attribution Non-Commercial S...,761240|761334,http://trace.boldsystems.org/traceIO/bold.org/...,2007-09-19 13:30:42|2007-09-19 15:01:55,Centre for Biodiversity Genomics|Centre for Bi...,F|R,LepF1|LepR1,COI-5P|COI-5P


In [34]:
## DNA downloader
missing_bolds = []
from urllib.request import urlopen
from time import sleep
f = open("log.txt", "w",buffering=1)

for index, bold_id in enumerate(datamat['bold_ids']):
    url = f"http://v3.boldsystems.org/index.php/API_Public/sequence?ids={bold_id[0][0]}&format=tsv"
    if index % 100 == 0:
        f.write(f"a posto index={index}")
        f.flush()
    try: 
        with urlopen(url) as response:
            body = str(response.read())
            nucleotide = body.split('\\r\\n')[1]
            #print(nucleotide)
            #print(bold_id)
            df.loc[df['processid']==bold_id[0][0],'nucleotide']=nucleotide 
      
    except Exception as e:
        #print('bad gate',index,e)
        #traceback.print_exc()
        f.write(""+str(index)+"\n")
        sleep(0.5)
        missing_bolds.append(bold_id)
        
    


In [36]:
df.loc[df['processid']=='FBCOA785-10']
len(missing_bolds)
df.head()

,processid,sampleid,recordID,catalognum,fieldnum,institution_storing,bin_uri,phylum_taxID,phylum_name,class_taxID,...,image_urls,copyright_licenses,trace_ids,trace_links,run_dates,sequencing_centers,directions,seq_primers,marker_codes,nucleotide
0,ANICL062-10,10ANIC-08059,1652746,10ANIC-08059,,Australian National Insect Collection,BOLD:AAE3294,20,Arthropoda,82,...,http://v3.boldsystems.org/pics/ANICL/10ANIC_08...,CreativeCommons - Attribution,2603931|2603908|2603881|2603843,http://trace.boldsystems.org/traceIO/bold.org/...,2010-12-16 00:05:06|2010-12-15 23:19:19|2010-1...,Centre for Biodiversity Genomics|Centre for Bi...,R|F|F|R,LepR1|MLepF1|LepF1|MLepR1,COI-5P|COI-5P|COI-5P|COI-5P,TACATTATATTTTATTTTTGGAATTTGAGCTGGTATAGTTGGAACT...
1,BCHYM11106-15,BC ZSM HYM 24216,6295592,BC ZSM HYM 24216,,"SNSB, Zoologische Staatssammlung Muenchen",BOLD:AAM4014,20,Arthropoda,82,...,http://v3.boldsystems.org/pics/BCHYM/BC_ZSM_HY...,CreativeCommons - Attribution Non-Commercial S...,8474808|8474713,http://trace.boldsystems.org/traceIO/bold.org/...,2015-10-05 14:30:49|2015-10-06 05:32:19,Centre for Biodiversity Genomics|Centre for Bi...,R|F,LepR1|LepF1,COI-5P|COI-5P,NaN
2,BCHYM7643-15,BC-ZSM-HYM-27761-D07,5806393,BC-ZSM-HYM-27761-D07,BC-ZSM-HYM-27761-D07,"SNSB, Zoologische Staatssammlung Muenchen",BOLD:AAZ8135,20,Arthropoda,82,...,http://v3.boldsystems.org/pics/BCHYM/BC-ZSM-HY...,CreativeCommons - Attribution Non-Commercial S...,7816517|7816422,http://trace.boldsystems.org/traceIO/bold.org/...,2015-05-04 11:31:35|2015-05-04 12:34:37,Centre for Biodiversity Genomics|Centre for Bi...,F|R,LepF1|LepR1,COI-5P|COI-5P,AGTTTTATATTTTATTTTTGGAATATGAGCAGGAATAATTGGATCA...
3,BLPBC407-07,06-SRNP-109430,568126,,06-SRNP-109430,Area de Conservacion Guanacaste,BOLD:AAA3052,20,Arthropoda,82,...,http://v3.boldsystems.org/pics/BLPBC/06-SRNP-1...,CreativeCommons - Attribution Non-Commercial S...,742969|742875,http://trace.boldsystems.org/traceIO/bold.org/...,2007-08-30 07:31:59|2007-08-30 06:00:49,Centre for Biodiversity Genomics|Centre for Bi...,R|F,LepR1|LepF1,COI-5P|COI-5P,--AACTTTATATTTCATTTTTGGTATTTGAGCAGGTATAGTAGGAA...
4,BLPBE439-07,07-SRNP-101378,623757,,07-SRNP-101378,Area de Conservacion Guanacaste,BOLD:AAA3052,20,Arthropoda,82,...,http://v3.boldsystems.org/pics/BLPBE/07-SRNP-1...,CreativeCommons - Attribution Non-Commercial S...,761240|761334,http://trace.boldsystems.org/traceIO/bold.org/...,2007-09-19 13:30:42|2007-09-19 15:01:55,Centre for Biodiversity Genomics|Centre for Bi...,F|R,LepF1|LepR1,COI-5P|COI-5P,--AACTTTATATTTCATTTTTGGTATTTGAGCAGGTATAGTAGGAA...


In [49]:
missing_bolds_4= []
for index, bold_id in enumerate(missing_bolds_3):
    url = f"http://v3.boldsystems.org/index.php/API_Public/sequence?ids={bold_id[0][0]}&format=tsv"
    if index % 100 == 0:
        f.write(f"a posto index={index}")
        f.flush()
    try: 
        with urlopen(url) as response:
            body = str(response.read())
            nucleotide = body.split('\\r\\n')[1]
            #print(nucleotide)
            #print(bold_id)
            df.loc[df['processid']==bold_id[0][0],'nucleotide']=nucleotide 
      
    except Exception as e:
        #print('bad gate',index,e)
        #traceback.print_exc()
        f.write(""+str(index)+"\n")
        sleep(0.5)
        missing_bolds_4.append(bold_id)

In [37]:
df.to_csv('partial.csv')

In [56]:
df['nucleotide'].isnull().sum()

5

In [57]:
df.to_csv("only_five_missing.csv")

In [59]:
df2= pd.read_csv("only_five_missing.csv")

In [93]:
import dill
dill.dump_session('notebook_env.db')